In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

In [5]:
df = pd.read_csv('/content/drive/MyDrive/UpdatedResumeDataSet.csv')

In [6]:
df

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."
...,...,...
957,Testing,Computer Skills: â¢ Proficient in MS office (...
958,Testing,â Willingness to accept the challenges. â ...
959,Testing,"PERSONAL SKILLS â¢ Quick learner, â¢ Eagerne..."
960,Testing,COMPUTER SKILLS & SOFTWARE KNOWLEDGE MS-Power ...


In [7]:
pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 50.1 MB/s eta 0:00:00


# Logistic regression

In [8]:
# train_ml_model.py



# Clean Text (basic cleaning)
def clean_text(text):
    return text.replace('\n', ' ').replace('\r', ' ').strip()

df['Resume'] = df['Resume'].apply(clean_text)

# Encode labels
le = LabelEncoder()
df['Category_encoded'] = le.fit_transform(df['Category'])

# Save label encoder
with open('/content/drive/MyDrive/label_encoder_ml.pkl', 'wb') as f:
    pickle.dump(le, f)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(df['Resume'], df['Category_encoded'], test_size=0.2, random_state=42)

# TF-IDF Vectorization
tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# Train Logistic Regression
model_logistic = LogisticRegression(max_iter=1000)
model_logistic.fit(X_train_tfidf, y_train)

# Save model and vectorizer
with open('/content/drive/MyDrive/ml_model_logistic.pkl', 'wb') as f:
    pickle.dump(model_logistic, f)

with open('/content/drive/MyDrive/tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf, f)

print("✅ ML Model trained and saved successfully.")


✅ ML Model trained and saved successfully.


In [9]:
# After predicting and calculating accuracy
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report
import matplotlib.pyplot as plt
# Predict on test data
y_pred = model_logistic.predict(X_test_tfidf)

# Calculate Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"✅ ML Model Accuracy on Test Set: {accuracy*100:.2f}%")
# Precision, Recall, F1
print("🔵 Classification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))


✅ ML Model Accuracy on Test Set: 99.48%
🔵 Classification Report:
                           precision    recall  f1-score   support

                 Advocate       1.00      1.00      1.00         3
                     Arts       1.00      1.00      1.00         6
       Automation Testing       1.00      1.00      1.00         5
               Blockchain       1.00      1.00      1.00         7
         Business Analyst       1.00      1.00      1.00         4
           Civil Engineer       1.00      1.00      1.00         9
             Data Science       1.00      1.00      1.00         5
                 Database       1.00      1.00      1.00         8
          DevOps Engineer       1.00      0.93      0.96        14
         DotNet Developer       1.00      1.00      1.00         5
            ETL Developer       1.00      1.00      1.00         7
   Electrical Engineering       1.00      1.00      1.00         6
                       HR       1.00      1.00      1.00       

In [ ]:
pip install PyMuPDF


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 45.8 MB/s eta 0:00:00


# logistic regression (esha prediction)

In [ ]:
# predict.py

import fitz  # PyMuPDF
import pickle
import torch
import numpy as np

# Load ML model and TF-IDF vectorizer
with open('/content/drive/MyDrive/ml_model_logistic.pkl', 'rb') as f:
    ml_model = pickle.load(f)

with open('/content/drive/MyDrive/tfidf_vectorizer.pkl', 'rb') as f:
    tfidf = pickle.load(f)

with open('/content/drive/MyDrive/label_encoder_ml.pkl', 'rb') as f:
    label_encoder_ml = pickle.load(f)

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    doc = fitz.open(pdf_path)
    for page in doc:
        text += page.get_text()
    return text

# Predict function
def predict_resume(pdf_path):
    text = extract_text_from_pdf(pdf_path)

    # ML Prediction
    text_tfidf = tfidf.transform([text])
    ml_probs = ml_model.predict_proba(text_tfidf)[0]
    ml_classes = label_encoder_ml.inverse_transform(np.arange(len(ml_probs)))

    # Convert to native float and append '%' sign
    ml_output = {cls: f"{float(prob * 100):.2f}%" for cls, prob in zip(ml_classes, ml_probs)}

    return ml_output

# Example Test
if __name__ == "__main__":
    pdf_file = '/content/drive/MyDrive/Esha Kunder:NNM22AD018.pdf'  # replace with your PDF file
    ml_result = predict_resume(pdf_file)
    print("🔵 Machine Learning Prediction (Normalized %):")
    for role, prob in ml_result.items():
        print(f"{role}: {prob}")


🔵 Machine Learning Prediction (Normalized %):
Advocate: 2.14%
Arts: 3.27%
Automation Testing: 2.50%
Blockchain: 3.93%
Business Analyst: 2.34%
Civil Engineer: 1.77%
Data Science: 29.53%
Database: 3.31%
DevOps Engineer: 2.75%
DotNet Developer: 2.41%
ETL Developer: 2.59%
Electrical Engineering: 2.61%
HR: 3.10%
Hadoop: 3.89%
Health and fitness: 2.76%
Java Developer: 4.35%
Mechanical Engineer: 3.35%
Network Security Engineer: 2.07%
Operations Manager: 2.25%
PMO: 2.05%
Python Developer: 5.85%
SAP Developer: 2.38%
Sales: 2.22%
Testing: 3.04%
Web Designing: 3.54%


# logistic regression (jyothi prediction)

In [11]:
# predict.py

import fitz  # PyMuPDF
import pickle
import torch
import numpy as np

# Load ML model and TF-IDF vectorizer
with open('/content/drive/MyDrive/ml_model_logistic.pkl', 'rb') as f:
    ml_model = pickle.load(f)

with open('/content/drive/MyDrive/tfidf_vectorizer.pkl', 'rb') as f:
    tfidf = pickle.load(f)

with open('/content/drive/MyDrive/label_encoder_ml.pkl', 'rb') as f:
    label_encoder_ml = pickle.load(f)

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    doc = fitz.open(pdf_path)
    for page in doc:
        text += page.get_text()
    return text

# Predict function
def predict_resume(pdf_path):
    text = extract_text_from_pdf(pdf_path)

    # ML Prediction
    text_tfidf = tfidf.transform([text])
    ml_probs = ml_model.predict_proba(text_tfidf)[0]
    ml_classes = label_encoder_ml.inverse_transform(np.arange(len(ml_probs)))

    # Convert to native float and append '%' sign
    ml_output = {cls: f"{float(prob * 100):.2f}%" for cls, prob in zip(ml_classes, ml_probs)}

    return ml_output

# Example Test
if __name__ == "__main__":
    pdf_file = '/content/drive/MyDrive/Copy of Resume.pdf'  # replace with your PDF file
    ml_result = predict_resume(pdf_file)
    print("🔵 Machine Learning Prediction (Normalized %):")
    for role, prob in ml_result.items():
        print(f"{role}: {prob}")


🔵 Machine Learning Prediction (Normalized %):
Advocate: 3.62%
Arts: 6.87%
Automation Testing: 3.11%
Blockchain: 5.10%
Business Analyst: 3.25%
Civil Engineer: 2.43%
Data Science: 6.26%
Database: 3.31%
DevOps Engineer: 2.76%
DotNet Developer: 2.55%
ETL Developer: 3.13%
Electrical Engineering: 4.10%
HR: 6.51%
Hadoop: 2.78%
Health and fitness: 4.05%
Java Developer: 4.88%
Mechanical Engineer: 5.93%
Network Security Engineer: 2.99%
Operations Manager: 2.05%
PMO: 2.10%
Python Developer: 6.54%
SAP Developer: 2.96%
Sales: 3.44%
Testing: 5.04%
Web Designing: 4.23%


# logistic regression (khyathi prediction)

In [10]:
# predict.py

import fitz  # PyMuPDF
import pickle
import torch
import numpy as np

# Load ML model and TF-IDF vectorizer
with open('/content/drive/MyDrive/ml_model_logistic.pkl', 'rb') as f:
    ml_model = pickle.load(f)

with open('/content/drive/MyDrive/tfidf_vectorizer.pkl', 'rb') as f:
    tfidf = pickle.load(f)

with open('/content/drive/MyDrive/label_encoder_ml.pkl', 'rb') as f:
    label_encoder_ml = pickle.load(f)

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    doc = fitz.open(pdf_path)
    for page in doc:
        text += page.get_text()
    return text

# Predict function
def predict_resume(pdf_path):
    text = extract_text_from_pdf(pdf_path)

    # ML Prediction
    text_tfidf = tfidf.transform([text])
    ml_probs = ml_model.predict_proba(text_tfidf)[0]
    ml_classes = label_encoder_ml.inverse_transform(np.arange(len(ml_probs)))

    # Convert to native float and append '%' sign
    ml_output = {cls: f"{float(prob * 100):.2f}%" for cls, prob in zip(ml_classes, ml_probs)}

    return ml_output

# Example Test
if __name__ == "__main__":
    pdf_file = '/content/drive/MyDrive/SHETTY KHYATHI RAJESH_NNM22AD050.pdf'  # replace with your PDF file
    ml_result = predict_resume(pdf_file)
    print("🔵 Machine Learning Prediction (Normalized %):")
    for role, prob in ml_result.items():
        print(f"{role}: {prob}")


🔵 Machine Learning Prediction (Normalized %):
Advocate: 2.99%
Arts: 5.82%
Automation Testing: 3.43%
Blockchain: 5.22%
Business Analyst: 3.78%
Civil Engineer: 2.62%
Data Science: 6.56%
Database: 3.72%
DevOps Engineer: 3.75%
DotNet Developer: 3.29%
ETL Developer: 3.77%
Electrical Engineering: 2.62%
HR: 4.04%
Hadoop: 3.97%
Health and fitness: 3.69%
Java Developer: 5.71%
Mechanical Engineer: 5.00%
Network Security Engineer: 3.04%
Operations Manager: 3.52%
PMO: 3.09%
Python Developer: 4.08%
SAP Developer: 3.01%
Sales: 3.03%
Testing: 5.28%
Web Designing: 4.98%


# Random forest

```

```



In [12]:
import pandas as pd
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report


# Clean Text
def clean_text(text):
    return text.replace('\n', ' ').replace('\r', ' ').strip()

df['Resume'] = df['Resume'].apply(clean_text)

# Encode Labels
le = LabelEncoder()
df['Category_encoded'] = le.fit_transform(df['Category'])

# Save Label Encoder
with open('/content/drive/MyDrive/label_encoder_rf.pkl', 'wb') as f:
    pickle.dump(le, f)

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(df['Resume'], df['Category_encoded'], test_size=0.2, random_state=42)

# TF-IDF Vectorization
tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# Train Random Forest
model_rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_rf.fit(X_train_tfidf, y_train)

# Save model and vectorizer
with open('/content/drive/MyDrive/ml_model_rf.pkl', 'wb') as f:
    pickle.dump(model_rf, f)

with open('/content/drive/MyDrive/tfidf_vectorizer_rf.pkl', 'wb') as f:
    pickle.dump(tfidf, f)

print("✅ Random Forest ML Model trained and saved successfully.")

# Predict on Test Data
y_pred = model_rf.predict(X_test_tfidf)

# Calculate Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"✅ Random Forest Model Accuracy on Test Set: {accuracy*100:.2f}%")

# Precision, Recall, F1
print("🔵 Classification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))


✅ Random Forest ML Model trained and saved successfully.
✅ Random Forest Model Accuracy on Test Set: 99.48%
🔵 Classification Report:
                           precision    recall  f1-score   support

                 Advocate       1.00      1.00      1.00         3
                     Arts       1.00      1.00      1.00         6
       Automation Testing       1.00      1.00      1.00         5
               Blockchain       1.00      1.00      1.00         7
         Business Analyst       1.00      1.00      1.00         4
           Civil Engineer       1.00      1.00      1.00         9
             Data Science       1.00      1.00      1.00         5
                 Database       1.00      1.00      1.00         8
          DevOps Engineer       1.00      0.93      0.96        14
         DotNet Developer       1.00      1.00      1.00         5
            ETL Developer       1.00      1.00      1.00         7
   Electrical Engineering       1.00      1.00      1.00      

# random forest esha prediction

In [ ]:
import fitz  # PyMuPDF
import pickle
import numpy as np

# Load Random Forest model and TF-IDF vectorizer
with open('/content/drive/MyDrive/ml_model_rf.pkl', 'rb') as f:
    rf_model = pickle.load(f)

with open('/content/drive/MyDrive/tfidf_vectorizer_rf.pkl', 'rb') as f:
    tfidf = pickle.load(f)

with open('/content/drive/MyDrive/label_encoder_rf.pkl', 'rb') as f:
    label_encoder_rf = pickle.load(f)

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    doc = fitz.open(pdf_path)
    for page in doc:
        text += page.get_text()
    return text

# Predict Function
def predict_resume_rf(pdf_path):
    text = extract_text_from_pdf(pdf_path)

    # Random Forest Prediction
    text_tfidf = tfidf.transform([text])
    rf_probs = rf_model.predict_proba(text_tfidf)[0]
    rf_classes = label_encoder_rf.inverse_transform(np.arange(len(rf_probs)))

    # Convert to native float and append '%' sign
    rf_output = {cls: f"{float(prob * 100):.2f}%" for cls, prob in zip(rf_classes, rf_probs)}

    return rf_output

# Example Test
if __name__ == "__main__":
    pdf_file = '/content/drive/MyDrive/Esha Kunder:NNM22AD018.pdf'  # Replace with your PDF
    rf_result = predict_resume_rf(pdf_file)
    print("🔵 Random Forest Prediction (Normalized %):")
    for role, prob in rf_result.items():
        print(f"{role}: {prob}")


🔵 Random Forest Prediction (Normalized %):
Advocate: 2.00%
Arts: 3.00%
Automation Testing: 2.00%
Blockchain: 6.00%
Business Analyst: 1.00%
Civil Engineer: 4.00%
Data Science: 41.00%
Database: 5.00%
DevOps Engineer: 4.00%
DotNet Developer: 2.00%
ETL Developer: 1.00%
Electrical Engineering: 2.00%
HR: 1.00%
Hadoop: 0.00%
Health and fitness: 2.00%
Java Developer: 2.00%
Mechanical Engineer: 2.00%
Network Security Engineer: 0.00%
Operations Manager: 2.00%
PMO: 1.00%
Python Developer: 10.00%
SAP Developer: 4.00%
Sales: 2.00%
Testing: 1.00%
Web Designing: 0.00%


# random forest (khyathi prediction)

In [13]:
import fitz  # PyMuPDF
import pickle
import numpy as np

# Load Random Forest model and TF-IDF vectorizer
with open('/content/drive/MyDrive/ml_model_rf.pkl', 'rb') as f:
    rf_model = pickle.load(f)

with open('/content/drive/MyDrive/tfidf_vectorizer_rf.pkl', 'rb') as f:
    tfidf = pickle.load(f)

with open('/content/drive/MyDrive/label_encoder_rf.pkl', 'rb') as f:
    label_encoder_rf = pickle.load(f)

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    doc = fitz.open(pdf_path)
    for page in doc:
        text += page.get_text()
    return text

# Predict Function
def predict_resume_rf(pdf_path):
    text = extract_text_from_pdf(pdf_path)

    # Random Forest Prediction
    text_tfidf = tfidf.transform([text])
    rf_probs = rf_model.predict_proba(text_tfidf)[0]
    rf_classes = label_encoder_rf.inverse_transform(np.arange(len(rf_probs)))

    # Convert to native float and append '%' sign
    rf_output = {cls: f"{float(prob * 100):.2f}%" for cls, prob in zip(rf_classes, rf_probs)}

    return rf_output

# Example Test
if __name__ == "__main__":
    pdf_file = '/content/drive/MyDrive/SHETTY KHYATHI RAJESH_NNM22AD050.pdf'  # Replace with your PDF
    rf_result = predict_resume_rf(pdf_file)
    print("🔵 Random Forest Prediction (Normalized %):")
    for role, prob in rf_result.items():
        print(f"{role}: {prob}")


🔵 Random Forest Prediction (Normalized %):
Advocate: 6.00%
Arts: 3.00%
Automation Testing: 5.00%
Blockchain: 4.00%
Business Analyst: 2.00%
Civil Engineer: 0.00%
Data Science: 13.00%
Database: 5.00%
DevOps Engineer: 0.00%
DotNet Developer: 5.00%
ETL Developer: 2.00%
Electrical Engineering: 3.00%
HR: 3.00%
Hadoop: 2.00%
Health and fitness: 6.00%
Java Developer: 0.00%
Mechanical Engineer: 14.00%
Network Security Engineer: 1.00%
Operations Manager: 1.00%
PMO: 0.00%
Python Developer: 2.00%
SAP Developer: 4.00%
Sales: 5.00%
Testing: 8.00%
Web Designing: 6.00%


# random forest jyothi prediction

In [14]:
import fitz  # PyMuPDF
import pickle
import numpy as np

# Load Random Forest model and TF-IDF vectorizer
with open('/content/drive/MyDrive/ml_model_rf.pkl', 'rb') as f:
    rf_model = pickle.load(f)

with open('/content/drive/MyDrive/tfidf_vectorizer_rf.pkl', 'rb') as f:
    tfidf = pickle.load(f)

with open('/content/drive/MyDrive/label_encoder_rf.pkl', 'rb') as f:
    label_encoder_rf = pickle.load(f)

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    doc = fitz.open(pdf_path)
    for page in doc:
        text += page.get_text()
    return text

# Predict Function
def predict_resume_rf(pdf_path):
    text = extract_text_from_pdf(pdf_path)

    # Random Forest Prediction
    text_tfidf = tfidf.transform([text])
    rf_probs = rf_model.predict_proba(text_tfidf)[0]
    rf_classes = label_encoder_rf.inverse_transform(np.arange(len(rf_probs)))

    # Convert to native float and append '%' sign
    rf_output = {cls: f"{float(prob * 100):.2f}%" for cls, prob in zip(rf_classes, rf_probs)}

    return rf_output

# Example Test
if __name__ == "__main__":
    pdf_file = '/content/drive/MyDrive/Copy of Resume.pdf'  # Replace with your PDF
    rf_result = predict_resume_rf(pdf_file)
    print("🔵 Random Forest Prediction (Normalized %):")
    for role, prob in rf_result.items():
        print(f"{role}: {prob}")


🔵 Random Forest Prediction (Normalized %):
Advocate: 8.00%
Arts: 10.00%
Automation Testing: 4.00%
Blockchain: 8.00%
Business Analyst: 1.00%
Civil Engineer: 3.00%
Data Science: 20.00%
Database: 2.00%
DevOps Engineer: 1.00%
DotNet Developer: 0.00%
ETL Developer: 2.00%
Electrical Engineering: 7.00%
HR: 6.00%
Hadoop: 0.00%
Health and fitness: 3.00%
Java Developer: 1.00%
Mechanical Engineer: 6.00%
Network Security Engineer: 0.00%
Operations Manager: 1.00%
PMO: 0.00%
Python Developer: 8.00%
SAP Developer: 5.00%
Sales: 1.00%
Testing: 0.00%
Web Designing: 3.00%


#decision tree

In [15]:
import pandas as pd
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report


# Clean Text
def clean_text(text):
    return text.replace('\n', ' ').replace('\r', ' ').strip()

df['Resume'] = df['Resume'].apply(clean_text)

# Encode Labels
le = LabelEncoder()
df['Category_encoded'] = le.fit_transform(df['Category'])

# Save Label Encoder
with open('/content/drive/MyDrive/label_encoder_dt.pkl', 'wb') as f:
    pickle.dump(le, f)

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(df['Resume'], df['Category_encoded'], test_size=0.2, random_state=42)

# TF-IDF Vectorization
tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# Train Decision Tree
model_dt = DecisionTreeClassifier(random_state=42)
model_dt.fit(X_train_tfidf, y_train)

# Save model and vectorizer
with open('/content/drive/MyDrive/ml_model_dt.pkl', 'wb') as f:
    pickle.dump(model_dt, f)

with open('/content/drive/MyDrive/tfidf_vectorizer_dt.pkl', 'wb') as f:
    pickle.dump(tfidf, f)

print("✅ Decision Tree ML Model trained and saved successfully.")

# Predict on Test Data
y_pred = model_dt.predict(X_test_tfidf)

# Calculate Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"✅ Decision Tree Model Accuracy on Test Set: {accuracy*100:.2f}%")

# Precision, Recall, F1
print("🔵 Classification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))


✅ Decision Tree ML Model trained and saved successfully.
✅ Decision Tree Model Accuracy on Test Set: 99.48%
🔵 Classification Report:
                           precision    recall  f1-score   support

                 Advocate       1.00      1.00      1.00         3
                     Arts       1.00      1.00      1.00         6
       Automation Testing       1.00      1.00      1.00         5
               Blockchain       1.00      1.00      1.00         7
         Business Analyst       1.00      1.00      1.00         4
           Civil Engineer       1.00      1.00      1.00         9
             Data Science       1.00      1.00      1.00         5
                 Database       1.00      1.00      1.00         8
          DevOps Engineer       1.00      0.93      0.96        14
         DotNet Developer       1.00      1.00      1.00         5
            ETL Developer       1.00      1.00      1.00         7
   Electrical Engineering       1.00      1.00      1.00      

In [ ]:
import fitz  # PyMuPDF
import pickle
import numpy as np

# Load Decision Tree model and TF-IDF vectorizer
with open('/content/drive/MyDrive/ml_model_dt.pkl', 'rb') as f:
    dt_model = pickle.load(f)

with open('/content/drive/MyDrive/tfidf_vectorizer_dt.pkl', 'rb') as f:
    tfidf = pickle.load(f)

with open('/content/drive/MyDrive/label_encoder_dt.pkl', 'rb') as f:
    label_encoder_dt = pickle.load(f)

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    doc = fitz.open(pdf_path)
    for page in doc:
        text += page.get_text()
    return text

# Predict Function
def predict_resume_dt(pdf_path):
    text = extract_text_from_pdf(pdf_path)

    # Decision Tree Prediction
    text_tfidf = tfidf.transform([text])
    dt_probs = dt_model.predict_proba(text_tfidf)[0]
    dt_classes = label_encoder_dt.inverse_transform(np.arange(len(dt_probs)))

    # Convert to native float and append '%' sign
    dt_output = {cls: f"{float(prob * 100):.2f}%" for cls, prob in zip(dt_classes, dt_probs)}

    return dt_output

# Example Test
if __name__ == "__main__":
    pdf_file = '/content/drive/MyDrive/Esha Kunder:NNM22AD018.pdf'  # Replace with your PDF
    dt_result = predict_resume_dt(pdf_file)
    print("🔵 Decision Tree Prediction (Normalized %):")
    for role, prob in dt_result.items():
        print(f"{role}: {prob}")


🔵 Decision Tree Prediction (Normalized %):
Advocate: 0.00%
Arts: 0.00%
Automation Testing: 0.00%
Blockchain: 0.00%
Business Analyst: 0.00%
Civil Engineer: 0.00%
Data Science: 100.00%
Database: 0.00%
DevOps Engineer: 0.00%
DotNet Developer: 0.00%
ETL Developer: 0.00%
Electrical Engineering: 0.00%
HR: 0.00%
Hadoop: 0.00%
Health and fitness: 0.00%
Java Developer: 0.00%
Mechanical Engineer: 0.00%
Network Security Engineer: 0.00%
Operations Manager: 0.00%
PMO: 0.00%
Python Developer: 0.00%
SAP Developer: 0.00%
Sales: 0.00%
Testing: 0.00%
Web Designing: 0.00%


#svm

In [16]:
import pandas as pd
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report



# Clean Text
def clean_text(text):
    return text.replace('\n', ' ').replace('\r', ' ').strip()

df['Resume'] = df['Resume'].apply(clean_text)

# Encode Labels
le = LabelEncoder()
df['Category_encoded'] = le.fit_transform(df['Category'])

# Save Label Encoder
with open('/content/drive/MyDrive/label_encoder_svm.pkl', 'wb') as f:
    pickle.dump(le, f)

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(df['Resume'], df['Category_encoded'], test_size=0.2, random_state=42)

# TF-IDF Vectorization
tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# Train SVM (with probability=True for predict_proba)
model_svm = SVC(kernel='linear', probability=True, random_state=42)
model_svm.fit(X_train_tfidf, y_train)

# Save model and vectorizer
with open('/content/drive/MyDrive/ml_model_svm.pkl', 'wb') as f:
    pickle.dump(model_svm, f)

with open('/content/drive/MyDrive/tfidf_vectorizer_svm.pkl', 'wb') as f:
    pickle.dump(tfidf, f)

print("✅ SVM ML Model trained and saved successfully.")

# Predict on Test Data
y_pred = model_svm.predict(X_test_tfidf)

# Calculate Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"✅ SVM Model Accuracy on Test Set: {accuracy*100:.2f}%")

# Precision, Recall, F1
print("🔵 Classification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))


✅ SVM ML Model trained and saved successfully.
✅ SVM Model Accuracy on Test Set: 99.48%
🔵 Classification Report:
                           precision    recall  f1-score   support

                 Advocate       1.00      1.00      1.00         3
                     Arts       1.00      1.00      1.00         6
       Automation Testing       1.00      1.00      1.00         5
               Blockchain       1.00      1.00      1.00         7
         Business Analyst       1.00      1.00      1.00         4
           Civil Engineer       1.00      1.00      1.00         9
             Data Science       1.00      1.00      1.00         5
                 Database       1.00      1.00      1.00         8
          DevOps Engineer       1.00      0.93      0.96        14
         DotNet Developer       1.00      1.00      1.00         5
            ETL Developer       1.00      1.00      1.00         7
   Electrical Engineering       1.00      1.00      1.00         6
               

# svm esha prediction

In [ ]:
import fitz  # PyMuPDF
import pickle
import numpy as np

# Load SVM model and TF-IDF vectorizer
with open('/content/drive/MyDrive/ml_model_svm.pkl', 'rb') as f:
    svm_model = pickle.load(f)

with open('/content/drive/MyDrive/tfidf_vectorizer_svm.pkl', 'rb') as f:
    tfidf = pickle.load(f)

with open('/content/drive/MyDrive/label_encoder_svm.pkl', 'rb') as f:
    label_encoder_svm = pickle.load(f)

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    doc = fitz.open(pdf_path)
    for page in doc:
        text += page.get_text()
    return text

# Predict Function
def predict_resume_svm(pdf_path):
    text = extract_text_from_pdf(pdf_path)

    # SVM Prediction
    text_tfidf = tfidf.transform([text])
    svm_probs = svm_model.predict_proba(text_tfidf)[0]
    svm_classes = label_encoder_svm.inverse_transform(np.arange(len(svm_probs)))

    # Convert to native float and append '%' sign
    svm_output = {cls: f"{float(prob * 100):.2f}%" for cls, prob in zip(svm_classes, svm_probs)}

    return svm_output

# Example Test
if __name__ == "__main__":
    pdf_file = '/content/drive/MyDrive/Esha Kunder:NNM22AD018.pdf'  # Replace with your PDF
    svm_result = predict_resume_svm(pdf_file)
    print("🔵 SVM Prediction (Normalized %):")
    for role, prob in svm_result.items():
        print(f"{role}: {prob}")


🔵 SVM Prediction (Normalized %):
Advocate: 2.15%
Arts: 3.13%
Automation Testing: 2.38%
Blockchain: 2.85%
Business Analyst: 2.44%
Civil Engineer: 1.04%
Data Science: 36.45%
Database: 3.65%
DevOps Engineer: 7.27%
DotNet Developer: 1.54%
ETL Developer: 1.74%
Electrical Engineering: 1.44%
HR: 3.58%
Hadoop: 2.28%
Health and fitness: 1.97%
Java Developer: 5.44%
Mechanical Engineer: 2.53%
Network Security Engineer: 1.13%
Operations Manager: 1.57%
PMO: 1.11%
Python Developer: 4.95%
SAP Developer: 1.81%
Sales: 1.40%
Testing: 3.43%
Web Designing: 2.72%


# svm khyathi prediction

In [18]:
import fitz  # PyMuPDF
import pickle
import numpy as np

# Load SVM model and TF-IDF vectorizer
with open('/content/drive/MyDrive/ml_model_svm.pkl', 'rb') as f:
    svm_model = pickle.load(f)

with open('/content/drive/MyDrive/tfidf_vectorizer_svm.pkl', 'rb') as f:
    tfidf = pickle.load(f)

with open('/content/drive/MyDrive/label_encoder_svm.pkl', 'rb') as f:
    label_encoder_svm = pickle.load(f)

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    doc = fitz.open(pdf_path)
    for page in doc:
        text += page.get_text()
    return text

# Predict Function
def predict_resume_svm(pdf_path):
    text = extract_text_from_pdf(pdf_path)

    # SVM Prediction
    text_tfidf = tfidf.transform([text])
    svm_probs = svm_model.predict_proba(text_tfidf)[0]
    svm_classes = label_encoder_svm.inverse_transform(np.arange(len(svm_probs)))

    # Convert to native float and append '%' sign
    svm_output = {cls: f"{float(prob * 100):.2f}%" for cls, prob in zip(svm_classes, svm_probs)}

    return svm_output

# Example Test
if __name__ == "__main__":
    pdf_file = '/content/drive/MyDrive/SHETTY KHYATHI RAJESH_NNM22AD050.pdf'  # Replace with your PDF
    svm_result = predict_resume_svm(pdf_file)
    print("🔵 SVM Prediction (Normalized %):")
    for role, prob in svm_result.items():
        print(f"{role}: {prob}")


🔵 SVM Prediction (Normalized %):
Advocate: 3.21%
Arts: 6.31%
Automation Testing: 3.71%
Blockchain: 3.80%
Business Analyst: 4.35%
Civil Engineer: 1.71%
Data Science: 8.27%
Database: 3.69%
DevOps Engineer: 10.81%
DotNet Developer: 2.25%
ETL Developer: 2.62%
Electrical Engineering: 1.46%
HR: 4.99%
Hadoop: 2.36%
Health and fitness: 2.82%
Java Developer: 9.10%
Mechanical Engineer: 4.07%
Network Security Engineer: 1.67%
Operations Manager: 2.62%
PMO: 1.76%
Python Developer: 3.45%
SAP Developer: 2.32%
Sales: 1.98%
Testing: 6.73%
Web Designing: 3.95%


# svm jyothi prediction

In [20]:
import fitz  # PyMuPDF
import pickle
import numpy as np

# Load SVM model and TF-IDF vectorizer
with open('/content/drive/MyDrive/ml_model_svm.pkl', 'rb') as f:
    svm_model = pickle.load(f)

with open('/content/drive/MyDrive/tfidf_vectorizer_svm.pkl', 'rb') as f:
    tfidf = pickle.load(f)

with open('/content/drive/MyDrive/label_encoder_svm.pkl', 'rb') as f:
    label_encoder_svm = pickle.load(f)

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    doc = fitz.open(pdf_path)
    for page in doc:
        text += page.get_text()
    return text

# Predict Function
def predict_resume_svm(pdf_path):
    text = extract_text_from_pdf(pdf_path)

    # SVM Prediction
    text_tfidf = tfidf.transform([text])
    svm_probs = svm_model.predict_proba(text_tfidf)[0]
    svm_classes = label_encoder_svm.inverse_transform(np.arange(len(svm_probs)))

    # Convert to native float and append '%' sign
    svm_output = {cls: f"{float(prob * 100):.2f}%" for cls, prob in zip(svm_classes, svm_probs)}

    return svm_output

# Example Test
if __name__ == "__main__":
    pdf_file = '/content/drive/MyDrive/Copy of Resume.pdf'  # Replace with your PDF
    svm_result = predict_resume_svm(pdf_file)
    print("🔵 SVM Prediction (Normalized %):")
    for role, prob in svm_result.items():
        print(f"{role}: {prob}")


🔵 SVM Prediction (Normalized %):
Advocate: 4.21%
Arts: 7.49%
Automation Testing: 3.35%
Blockchain: 3.92%
Business Analyst: 3.19%
Civil Engineer: 1.49%
Data Science: 7.52%
Database: 3.42%
DevOps Engineer: 7.83%
DotNet Developer: 1.67%
ETL Developer: 2.21%
Electrical Engineering: 2.49%
HR: 9.34%
Hadoop: 1.67%
Health and fitness: 3.22%
Java Developer: 6.93%
Mechanical Engineer: 4.93%
Network Security Engineer: 1.68%
Operations Manager: 1.46%
PMO: 1.17%
Python Developer: 5.85%
SAP Developer: 2.37%
Sales: 2.34%
Testing: 6.73%
Web Designing: 3.55%


# multi nomial nb

In [30]:
import pandas as pd
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report



# Clean Text
def clean_text(text):
    return text.replace('\n', ' ').replace('\r', ' ').strip()

df['Resume'] = df['Resume'].apply(clean_text)

# Encode Labels
le = LabelEncoder()
df['Category_encoded'] = le.fit_transform(df['Category'])

# Save Label Encoder
with open('/content/drive/MyDrive/label_encoder_nb.pkl', 'wb') as f:
    pickle.dump(le, f)

# TF-IDF Vectorization
tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(df['Resume'])
y = df['Category_encoded']

# Save TF-IDF
with open('/content/drive/MyDrive/tfidf_vectorizer_nb.pkl', 'wb') as f:
    pickle.dump(tfidf, f)

# ===============================
# 1. Cross-Validation
# ===============================
model_nb = MultinomialNB()

cv_scores = cross_val_score(model_nb, X, y, cv=5, scoring='accuracy')

print("✅ Cross-Validation Accuracy Scores:", cv_scores)
print(f"✅ Mean CV Accuracy: {cv_scores.mean()*100:.2f}%")

# ===============================
# 2. Train-Test Split and Test
# ===============================
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_nb.fit(X_train, y_train)

# Predict on Test Set
y_pred = model_nb.predict(X_test)

# Test Accuracy
test_accuracy = accuracy_score(y_test, y_pred)
print(f"✅ Test Set Accuracy: {test_accuracy*100:.2f}%")

# Classification Report
print("🔵 Classification Report on Test Set:")
print(classification_report(y_test, y_pred, target_names=le.classes_))

# ===============================
# 3. Save the trained model
# ===============================
with open('/content/drive/MyDrive/ml_model_nb.pkl', 'wb') as f:
    pickle.dump(model_nb, f)

print("✅ MultinomialNB Model trained and saved successfully.")


✅ Cross-Validation Accuracy Scores: [0.9119171  0.95336788 0.9375     0.94270833 0.96354167]
✅ Mean CV Accuracy: 94.18%
✅ Test Set Accuracy: 93.78%
🔵 Classification Report on Test Set:
                           precision    recall  f1-score   support

                 Advocate       0.00      0.00      0.00         3
                     Arts       1.00      1.00      1.00         6
       Automation Testing       1.00      1.00      1.00         5
               Blockchain       1.00      1.00      1.00         7
         Business Analyst       1.00      1.00      1.00         4
           Civil Engineer       1.00      0.56      0.71         9
             Data Science       1.00      1.00      1.00         5
                 Database       1.00      1.00      1.00         8
          DevOps Engineer       1.00      0.93      0.96        14
         DotNet Developer       1.00      0.80      0.89         5
            ETL Developer       1.00      1.00      1.00         7
   Electri

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [33]:
import fitz  # PyMuPDF
import pickle
import numpy as np

# Load MultinomialNB model and TF-IDF vectorizer
with open('/content/drive/MyDrive/ml_model_nb.pkl', 'rb') as f:
    nb_model = pickle.load(f)

with open('/content/drive/MyDrive/tfidf_vectorizer_nb.pkl', 'rb') as f:
    tfidf = pickle.load(f)

with open('/content/drive/MyDrive/label_encoder_nb.pkl', 'rb') as f:
    label_encoder_nb = pickle.load(f)

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    doc = fitz.open(pdf_path)
    for page in doc:
        text += page.get_text()
    return text

# Predict Function
def predict_resume_nb(pdf_path):
    text = extract_text_from_pdf(pdf_path)

    # Naive Bayes Prediction
    text_tfidf = tfidf.transform([text])
    nb_probs = nb_model.predict_proba(text_tfidf)[0]
    nb_classes = label_encoder_nb.inverse_transform(np.arange(len(nb_probs)))

    # Convert to native float and append '%' sign
    nb_output = {cls: f"{float(prob * 100):.2f}%" for cls, prob in zip(nb_classes, nb_probs)}

    return nb_output

# Example Test
if __name__ == "__main__":
    pdf_file = '/content/drive/MyDrive/SHETTY KHYATHI RAJESH_NNM22AD050.pdf'  # Replace with your PDF
    nb_result = predict_resume_nb(pdf_file)
    print("🔵 MultinomialNB Prediction (Normalized %):")
    for role, prob in nb_result.items():
        print(f"{role}: {prob}")


🔵 MultinomialNB Prediction (Normalized %):
Advocate: 0.88%
Arts: 3.20%
Automation Testing: 2.09%
Blockchain: 5.72%
Business Analyst: 2.73%
Civil Engineer: 0.95%
Data Science: 7.31%
Database: 2.58%
DevOps Engineer: 5.66%
DotNet Developer: 2.37%
ETL Developer: 4.06%
Electrical Engineering: 1.28%
HR: 2.17%
Hadoop: 5.94%
Health and fitness: 1.73%
Java Developer: 13.99%
Mechanical Engineer: 4.39%
Network Security Engineer: 1.71%
Operations Manager: 3.49%
PMO: 2.64%
Python Developer: 3.96%
SAP Developer: 1.23%
Sales: 2.64%
Testing: 8.60%
Web Designing: 8.69%


In [28]:
import pandas as pd
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report

# Clean Text
def clean_text(text):
    return text.replace('\n', ' ').replace('\r', ' ').strip()

df['Resume'] = df['Resume'].apply(clean_text)

# Encode Labels
le = LabelEncoder()
df['Category_encoded'] = le.fit_transform(df['Category'])

# Save Label Encoder
with open('/content/drive/MyDrive/label_encoder_gaussian_nb.pkl', 'wb') as f:
    pickle.dump(le, f)

# TF-IDF Vectorization
tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(df['Resume'])
y = df['Category_encoded']

# GaussianNB needs dense array
X_dense = X.toarray()

# Save TF-IDF
with open('/content/drive/MyDrive/tfidf_vectorizer_gaussian_nb.pkl', 'wb') as f:
    pickle.dump(tfidf, f)

# ===============================
# 1. Cross-Validation
# ===============================
model_gaussian_nb = GaussianNB()

cv_scores = cross_val_score(model_gaussian_nb, X_dense, y, cv=5, scoring='accuracy')

print("✅ Cross-Validation Accuracy Scores:", cv_scores)
print(f"✅ Mean CV Accuracy: {cv_scores.mean()*100:.2f}%")

# ===============================
# 2. Train-Test Split and Test
# ===============================
X_train, X_test, y_train, y_test = train_test_split(X_dense, y, test_size=0.2, random_state=42)

model_gaussian_nb.fit(X_train, y_train)

# Predict on Test Set
y_pred = model_gaussian_nb.predict(X_test)

# Test Accuracy
test_accuracy = accuracy_score(y_test, y_pred)
print(f"✅ Test Set Accuracy: {test_accuracy*100:.2f}%")

# Classification Report
print("🔵 Classification Report on Test Set:")
print(classification_report(y_test, y_pred, target_names=le.classes_))

# ===============================
# 3. Save the trained model
# ===============================
with open('/content/drive/MyDrive/ml_model_gaussian_nb.pkl', 'wb') as f:
    pickle.dump(model_gaussian_nb, f)

print("✅ GaussianNB Model trained and saved successfully.")


✅ Cross-Validation Accuracy Scores: [0.97927461 1.         1.         1.         1.        ]
✅ Mean CV Accuracy: 99.59%
✅ Test Set Accuracy: 99.48%
🔵 Classification Report on Test Set:
                           precision    recall  f1-score   support

                 Advocate       1.00      1.00      1.00         3
                     Arts       1.00      1.00      1.00         6
       Automation Testing       1.00      1.00      1.00         5
               Blockchain       1.00      1.00      1.00         7
         Business Analyst       1.00      1.00      1.00         4
           Civil Engineer       1.00      1.00      1.00         9
             Data Science       1.00      1.00      1.00         5
                 Database       0.89      1.00      0.94         8
          DevOps Engineer       1.00      0.93      0.96        14
         DotNet Developer       1.00      1.00      1.00         5
            ETL Developer       1.00      1.00      1.00         7
   Electri

In [29]:
import fitz  # PyMuPDF
import pickle
import numpy as np

# Load GaussianNB model and TF-IDF vectorizer
with open('/content/drive/MyDrive/ml_model_gaussian_nb.pkl', 'rb') as f:
    gaussian_nb_model = pickle.load(f)

with open('/content/drive/MyDrive/tfidf_vectorizer_gaussian_nb.pkl', 'rb') as f:
    tfidf = pickle.load(f)

with open('/content/drive/MyDrive/label_encoder_gaussian_nb.pkl', 'rb') as f:
    label_encoder_gaussian_nb = pickle.load(f)

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    doc = fitz.open(pdf_path)
    for page in doc:
        text += page.get_text()
    return text

# Predict Function
def predict_resume_gaussian_nb(pdf_path):
    text = extract_text_from_pdf(pdf_path)

    # Gaussian NB Prediction
    text_tfidf = tfidf.transform([text]).toarray()  # Convert to dense for GaussianNB
    nb_probs = gaussian_nb_model.predict_proba(text_tfidf)[0]
    nb_classes = label_encoder_gaussian_nb.inverse_transform(np.arange(len(nb_probs)))

    # Convert to native float and append '%' sign
    nb_output = {cls: f"{float(prob * 100):.2f}%" for cls, prob in zip(nb_classes, nb_probs)}

    return nb_output

# Example Test
if __name__ == "__main__":
    pdf_file = '/content/drive/MyDrive/Copy of Resume.pdf'  # Replace with your PDF
    nb_result = predict_resume_gaussian_nb(pdf_file)
    print("🔵 Gaussian Naive Bayes Prediction (Normalized %):")
    for role, prob in nb_result.items():
        print(f"{role}: {prob}")


🔵 Gaussian Naive Bayes Prediction (Normalized %):
Advocate: 0.00%
Arts: 0.00%
Automation Testing: 0.00%
Blockchain: 0.00%
Business Analyst: 0.00%
Civil Engineer: 0.00%
Data Science: 0.00%
Database: 0.00%
DevOps Engineer: 0.00%
DotNet Developer: 0.00%
ETL Developer: 0.00%
Electrical Engineering: 0.00%
HR: 0.00%
Hadoop: 0.00%
Health and fitness: 0.00%
Java Developer: 0.00%
Mechanical Engineer: 0.00%
Network Security Engineer: 100.00%
Operations Manager: 0.00%
PMO: 0.00%
Python Developer: 0.00%
SAP Developer: 0.00%
Sales: 0.00%
Testing: 0.00%
Web Designing: 0.00%
